In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [28]:
# 모델 정의
class PINN(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.linear1 = nn.Linear(2, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, 1)

    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=-1)
        outputs = torch.tanh(self.linear1(inputs))
        outputs = torch.tanh(self.linear2(outputs))
        outputs = self.linear3(outputs)
        return outputs

In [29]:
# u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
def gradient(y, x):
    return torch.autograd.grad(y, x, 
                               grad_outputs=torch.ones_like(y),
                               create_graph=True)[0]

# 손실 함수 정의
def loss_fn(model, x, t, alpha):
    x.requires_grad_(True)
    t.requires_grad_(True)

    u = model(x, t)
    u_t = gradient(u, t)
    u_x = gradient(u, x)
    u_xx = gradient(u_x, x)
    pde_loss = torch.mean((u_t - alpha * u_xx)**2)

    # 경계 조건 손실
    x_bc = torch.tensor([[0.0], [1.0]], requires_grad=False).to(device)
    t_bc = torch.tensor([[0.0], [0.0]], requires_grad=False).to(device)
    u_bc = model(x_bc, t_bc)
    bc_loss = torch.mean(u_bc**2)

    # 초기 조건 손실
    x_ic = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)
    t_ic = torch.zeros_like(x_ic).to(device)
    u_ic = model(x_ic, t_ic)
    ic_loss = torch.mean((u_ic - torch.sin(np.pi * x_ic))**2)

    loss = pde_loss + bc_loss + ic_loss
    return loss, (pde_loss, bc_loss, ic_loss)

In [30]:
# 랜덤 시드
torch.manual_seed(11)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습
alpha = 1.0
hidden_size = 20
model = PINN(hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

x = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)
t = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)

def train_step(model, optimizer, x, t, alpha):
    loss, aux = loss_fn(model, x, t, alpha)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss, aux

n_epochs = 10000
for epoch in range(1, n_epochs + 1):
    loss, aux = train_step(model, optimizer, x, t, alpha)
    pde_loss, bc_loss, ic_loss = aux

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch:5d}/{n_epochs}] Loss: {loss.item():.3e} "
              f"pde: {pde_loss.item():.3e} bc: {bc_loss.item():.3e} ic: {ic_loss.item():.3e}")

[ 1000/10000] Loss: 4.063e-04 pde: 2.204e-05 bc: 4.335e-05 ic: 3.409e-04
[ 2000/10000] Loss: 1.417e-04 pde: 1.309e-05 bc: 2.910e-05 ic: 9.950e-05
[ 3000/10000] Loss: 7.526e-05 pde: 4.550e-06 bc: 6.855e-06 ic: 6.385e-05
[ 4000/10000] Loss: 4.203e-05 pde: 3.298e-06 bc: 3.987e-06 ic: 3.474e-05
[ 5000/10000] Loss: 2.482e-05 pde: 3.458e-06 bc: 2.321e-06 ic: 1.904e-05
[ 6000/10000] Loss: 1.641e-05 pde: 3.225e-06 bc: 1.488e-06 ic: 1.169e-05
[ 7000/10000] Loss: 1.222e-05 pde: 2.838e-06 bc: 9.772e-07 ic: 8.406e-06
[ 8000/10000] Loss: 1.666e-04 pde: 1.491e-04 bc: 8.102e-06 ic: 9.456e-06
[ 9000/10000] Loss: 8.652e-06 pde: 2.034e-06 bc: 6.219e-07 ic: 5.997e-06
[10000/10000] Loss: 8.515e-06 pde: 2.402e-06 bc: 6.415e-07 ic: 5.471e-06
